In [ ]:
import pandas as pd

In [ ]:
# bidmc
ICDs = pd.read_csv('/home/bram/Project_PD/PD project/Cohort construction/initial classification/MEDICDDATA POS/BIDMC_ICDPLUS.csv')

In [ ]:
matrix=pd.read_csv('/home/bram/Project_PD/PD project/Feature matrix/01_Text_feature_matrix/feat_matrix_dropped.csv')

In [ ]:
print(len(matrix))
print(matrix.columns)

In [ ]:
print(ICDs.head())
print(len(ICDs))

In [ ]:
# import BIDMC 

# # example
# ICDs = pd.read_csv('/home/cdac-c-15/Desktop/NAXCA/Cohort All Files/allICDCodesMGB.csv')
# ICDs = ICDs[ICDs['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ShiftedContactDTS']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
# ICDs = ICDs[['BDSPPatientID', 'ICD Date']]
# ICDs.head()

ICDs = ICDs[ICDs['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
ICDs['ICD Date'] = pd.to_datetime(ICDs['CreateDate']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
ICDs = ICDs[['BDSPPatientID', 'ICD Date']]
ICDs.head()

In [ ]:
print(ICDs['BDSPPatientID'].nunique())

In [ ]:
print(len(ICDs))

In [ ]:
ICDs_MGB = pd.read_csv('/home/bram/Project_PD/PD project/Cohort construction/initial classification/MEDICDDATA POS/MGB_ICDPLUS.csv')
print(ICDs_MGB.head())
print(len(ICDs_MGB))

ICDs_MGB = ICDs_MGB[ICDs_MGB['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
ICDs_MGB['ICD Date'] = pd.to_datetime(ICDs_MGB['CreateDate']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
ICDs_MGB = ICDs_MGB[['BDSPPatientID', 'ICD Date']]
ICDs_MGB.head()
print(len(ICDs_MGB))

In [ ]:
print(ICDs_MGB.head())
print(len(ICDs_MGB))
print(ICDs_MGB['BDSPPatientID'].nunique()) #validated 964

In [ ]:
ICDs_STF = pd.read_csv('/home/bram/Project_PD/PD project/Cohort construction/initial classification/MEDICDDATA POS/STF_ICDPLUS.csv')
print(ICDs_STF.head())
print(len(ICDs_STF))


In [ ]:
ICDs_STF['BDSPPatientID']=ICDs_STF['bdsp_patient_id']
print(ICDs_STF.head())

In [ ]:
ICDs_STF = ICDs_STF[ICDs_STF['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
ICDs_STF['ICD Date'] = pd.to_datetime(ICDs_STF['CreateDate']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
ICDs_STF = ICDs_STF[['BDSPPatientID', 'ICD Date']]
ICDs_STF.head()
print(len(ICDs_STF))

In [ ]:
print(ICDs_STF['BDSPPatientID'].nunique())#validated icd+ are 363 patients
print(ICDs_STF.head())

In [ ]:
ICDs_compl=[ICDs,ICDs_STF ,ICDs_MGB]

ICDs_complete=pd.concat(ICDs_compl, ignore_index=True)

In [ ]:
print(ICDs_complete.head)
print(len(ICDs_complete))
print(ICDs_complete['BDSPPatientID'].nunique())

In [ ]:
matrix['CreateDate']= pd.to_datetime(matrix['CreateDate']) 
ICDs_complete['ICD Date'] = pd.to_datetime(ICDs_complete['ICD Date'])

In [ ]:
# add 1 or 0 to new column in matrix that denotes whether an icd is present +- 6 months from note creation date
#merge the dfs on patient ID
merged_df = pd.merge(matrix, ICDs_complete, on='BDSPPatientID')
#check for date timeline
merged_df['ICD'] = merged_df.apply(
    lambda row: (row['CreateDate'] >= row['ICD Date'] - pd.DateOffset(months=6)) and 
                (row['CreateDate'] <= row['ICD Date'] + pd.DateOffset(months=6)), axis=1)
matrix['ICD'] = matrix.apply(
    lambda row: merged_df[(merged_df['BDSPPatientID'] == row['BDSPPatientID']) & merged_df['ICD']].shape[0] > 0, axis=1).astype(int)

print(len(matrix))
matrix.head()

In [ ]:
print(matrix.sample(20))

In [ ]:
matrix.to_csv('matrix_incl_icd.csv', index=False)